# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [ ]:
# Заметки

# bigrams.txt
# Частота | Слово 1 | Слово 2
#
# 275  a    a
# 29   a    all

# coca_all_links
# Частота | Слово 1 | Слово 2 | Часть речи 1 | Часть речи 2
#
# 36  a-National  Rank    jj   nn1
# 92  abandoned   building  jj   nn1

# fivegrams.txt
# Частота | Слово 1 | Слово 2 | Слово 3 | Слово 4 | Слово 5
#
# 16  a    babe    in    the    woods
# 6   a    baby    at    her    breast


In [39]:
#Test bigrams

import nltk

nltk.download('words',quiet=True)
nltk.download('reuters',quiet=True)
nltk.download('punkt_tab',quiet=True)


def generate_candidates(word):
    abc = 'abcdefghijklmnopqrstuvwxyz'
    edits = set()

    for i in range(len(word)):
        edits.add(word[:i] + word[i+1:])
    for i in range(len(word)):
        for char in abc:
            edits.add(word[:i] + char + word[i+1:])
    for i in range(len(word) + 1):
        for char in abc:
            edits.add(word[:i] + char + word[i:])
    return edits & set(words.words())


def correct_sentence_bigram(sentence, bigram_freq):
    tokens = nltk.word_tokenize(sentence.lower())
    corrected_tokens = []

    for i in range(len(tokens)):
        word = tokens[i]
        if i > 0:
            prev_word = corrected_tokens[-1]
            candidates = generate_candidates(word)
            if candidates:
                best_candidate = max(candidates, key=lambda w: bigram_freq.get((prev_word, w), 1))
                corrected_tokens.append(best_candidate)
            else:
                corrected_tokens.append(word)
        else:
            corrected_tokens.append(word)

    return ' '.join(corrected_tokens)

bigram_freq = {}
with open("bigrams.txt", 'r', encoding='latin-1') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) == 3:
            freq, w1, w2 = parts
            bigram_freq[(w1, w2)] = int(freq)


sample_text = "I am dking sport every day"
corrected_text = correct_sentence_bigram(sample_text, bigram_freq)
print(corrected_text)


i am doing spor revery dag


In [50]:
#Test coca
import nltk
from nltk.corpus import reuters, words


nltk.download('words',quiet=True)
nltk.download('reuters',quiet=True)
nltk.download('punkt_tab',quiet=True)
word_list = set(words.words())

def generate_candidates(word):
    abc = 'abcdefghijklmnopqrstuvwxyz'
    edits = set()
    for i in range(len(word)):
        edits.add(word[:i] + word[i+1:])
    for i in range(len(word)):
        for char in abc:
            edits.add(word[:i] + char + word[i+1:])
    for i in range(len(word) + 1):
        for char in abc:
            edits.add(word[:i] + char + word[i:])
    valid_edits = {w for w in edits if w in word_list}
    return valid_edits if valid_edits else {word}

def correct_sentence_coca(sentence, coca_freq):
    tokens = nltk.word_tokenize(sentence.lower())
    corrected_tokens = []
    for i in range(len(tokens)):
        word = tokens[i]
        candidates = generate_candidates(word)
        if i > 0:
            prev_word = corrected_tokens[-1]
            best_candidate = max(candidates, key=lambda w: coca_freq.get((prev_word, w), 1), default=word)
        else:
            best_candidate = max(candidates, key=lambda w: w in word_list, default=word)
        corrected_tokens.append(best_candidate)
    return ' '.join(corrected_tokens)
coca_freq = {}
with open("coca_all_links.txt", 'r', encoding='latin-1') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) >= 3:
            freq, w1, w2 = parts[:3]
            coca_freq[(w1, w2)] = int(freq)

sample_text = "I am dking sport every day"
corrected_text = correct_sentence_coca(sample_text, coca_freq)
print(corrected_text)

c om eking spor revery dag


In [56]:
#Test fivegrams
import nltk
from nltk.corpus import words
from nltk.metrics import edit_distance

nltk.download('words', quiet=True)
nltk.download('punkt', quiet=True)
word_list = set(words.words())
def correct_word(word):
    if word in word_list:
        return word
    closest_word = min(word_list, key=lambda w: edit_distance(word, w))
    return closest_word
def correct_sentence_fivegram(sentence, fivegram_freq):
    tokens = nltk.word_tokenize(sentence)
    corrected_tokens = []
    for i, word in enumerate(tokens):
        word_lower = word.lower()
        original_case = word[0].isupper() if word else False

        context = tuple(corrected_tokens[max(0, i - 4):i])

        if context in fivegram_freq:
            candidates = fivegram_freq[context]
            best_candidate = max(candidates, key=candidates.get)
        else:
            best_candidate = correct_word(word_lower)
        corrected_tokens.append(best_candidate.capitalize() if original_case else best_candidate)

    return ' '.join(corrected_tokens)

fivegram_freq = {}
with open("fivegrams.txt", 'r', encoding='latin-1') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) == 6:
            freq, *words = parts
            freq = int(freq)
            context, target_word = tuple(words[:-1]), words[-1]
            if context not in fivegram_freq:
                fivegram_freq[context] = {}
            fivegram_freq[context][target_word] = freq

sample_text = "I am dking sport every day"
corrected_text = correct_sentence_fivegram(sample_text, fivegram_freq)
print(corrected_text)

I am ding sport every day


In [9]:
import re
import collections

def words(text):
    return re.findall(r'\w+', text)

def load_corpus(filename):
    with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        text = f.read()
    tokens = words(text)
    return tokens

def generate_ngrams(tokens, n):
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def count_ngrams(tokens, n):
    ngram_counts = collections.Counter(generate_ngrams(tokens, n))
    total = sum(ngram_counts.values())
    return {ngram: count / total for ngram, count in ngram_counts.items()}

def save_ngrams(ngram_counts, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for ngram, freq in ngram_counts.items():
            f.write(f"{ngram}\t{freq}\n")


tokens = load_corpus('big.txt')
unigram_counts = collections.Counter(tokens)
bigram_counts = count_ngrams(tokens, 2)
trigram_counts = count_ngrams(tokens, 3)
fourgram_counts = count_ngrams(tokens, 4)
fivegram_counts = count_ngrams(tokens, 5)

save_ngrams(unigram_counts, 'unigrams_big.txt')
save_ngrams(bigram_counts, 'bigrams_big.txt')
save_ngrams(trigram_counts, 'trigrams_big.txt')
save_ngrams(fourgram_counts, 'fourgrams_big.txt')
save_ngrams(fivegram_counts, 'fivegrams_big.txt')

In [10]:
def load_ngrams(filename):
    ngram_counts = {}
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            ngram, freq = line.strip().split('\t')
            ngram_counts[ngram] = float(freq)
    return ngram_counts


unigram_counts = load_ngrams('unigrams_big.txt')
bigram_counts = load_ngrams('bigrams_big.txt')
trigram_counts = load_ngrams('trigrams_big.txt')
fourgram_counts = load_ngrams('fourgrams_big.txt')
fivegram_counts = load_ngrams('fivegrams_big.txt')

In [11]:

def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words, word_dict):
    return set(w for w in words if w in word_dict)

In [12]:
def correct_word(word, unigram_counts, bigram_counts, prev_word):
    candidates = (known([word], unigram_counts) or
                  known(edits1(word), unigram_counts) or
                  known(edits2(word), unigram_counts) or
                  [word])

    if prev_word:
        return max(candidates, key=lambda w: bigram_counts.get(f"{prev_word} {w}", 0) + unigram_counts.get(w, 0))
    else:
        return max(candidates, key=lambda w: unigram_counts.get(w, 0))

def correct_sentence(sentence, unigram_counts, bigram_counts, trigram_counts, fivegram_counts):
    words_with_punctuation = re.findall(r'\b\w+\b|[.,!?;]', sentence)
    corrected_words = []

    for i, word in enumerate(words_with_punctuation):
        if re.match(r'\w+', word):
            prev_word = corrected_words[i-1] if i > 0 else None
            prev_prev_word = corrected_words[i-2] if i > 1 else None

            candidates = known([word.lower()], unigram_counts) or known(edits1(word.lower()), unigram_counts) or known(edits2(word.lower()), unigram_counts) or [word.lower()]

            def score(w):
                unigram_score = unigram_counts.get(w, 0)
                bigram_score = bigram_counts.get(f"{prev_word} {w}", 0) if prev_word else 0
                trigram_score = trigram_counts.get(f"{prev_prev_word} {prev_word} {w}", 0) if prev_prev_word else 0
                fivegram_score = fivegram_counts.get(f"{prev_prev_word} {prev_word} {w}", 0) if prev_prev_word else 0
                return unigram_score + 2 * bigram_score + 3 * trigram_score + 5 * fivegram_score

            best_candidate = max(candidates, key=score)
            if word.istitle():
                best_candidate = best_candidate.capitalize()
            corrected_words.append(best_candidate)
        else:
            corrected_words.append(word)

    return ' '.join(corrected_words)

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

*Your text here...*

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [6]:
import random

ERROR_TYPES = [
    "spelling", "homophone", "grammar", "word_order"
]

def introduce_errors(sentence):
    words = sentence.split()
    error_type = random.choice(ERROR_TYPES)

    if error_type == "spelling":
        if words:
            idx = random.randint(0, len(words) - 1)
            words[idx] = words[idx][:max(1, len(words[idx]) - 1)]
    elif error_type == "homophone":
        homophones = {"their": "there", "there": "their", "your": "you're", "you're": "your", "to": "too", "too": "to"}
        words = [homophones.get(w, w) for w in words]
    elif error_type == "grammar":
        if "is" in words:
            words[words.index("is")] = "are"
    elif error_type == "word_order":
        if len(words) > 1:
            words[0], words[1] = words[1], words[0]

    return " ".join(words)

#I took these sentences from different sites and partially generated them
correct_sentences = [
    "I am going to the store later.",
    "This is a simple test sentence.",
    "She enjoys playing the piano.",
    "The weather is really nice today.",
    "We need to finish our project soon.",
    "He quickly ran across the street to catch the bus.",
    "My favorite color is blue, but I also like red.",
    "The dog barked loudly at the stranger.",
    "We had a great time at the amusement park.",
    "She bought a new dress for the party.",
    "John loves reading books about history.",
    "The cat is sleeping on the sofa.",
    "We should leave early to avoid traffic.",
    "Can you help me carry these bags?",
    "Her voice was soft but clear.",
    "The sun rises in the east and sets in the west.",
    "This puzzle is difficult to solve.",
    "Our teacher gave us homework for the weekend.",
    "The museum has a collection of ancient artifacts.",
    "She practices yoga every morning.",
    "The train arrived at the station on time.",
    "My grandmother tells wonderful stories.",
    "Please turn off the lights before you leave.",
    "We enjoyed our trip to the mountains.",
    "He apologized for being late to the meeting.",
    "The new restaurant serves delicious pasta.",
    "His handwriting is very neat and clear.",
    "They adopted a cute little puppy.",
    "Her favorite subject in school is mathematics.",
    "We walked along the beach at sunset.",
    "The scientist explained the theory in detail.",
    "She is learning to play the violin.",
    "The movie was both exciting and emotional.",
    "We planted flowers in our garden last spring.",
    "The bakery sells fresh bread every morning.",
    "My brother is studying engineering at university.",
    "Their wedding was a beautiful ceremony.",
    "The library has a vast collection of books.",
    "I enjoy listening to classical music.",
    "The artist painted a stunning landscape.",
    "The city skyline looks amazing at night.",
    "Her dress was made of fine silk.",
    "We celebrated my birthday with a big party.",
    "The students are preparing for their exams.",
    "He built a wooden birdhouse for his garden.",
    "She loves watching the stars at night.",
    "The chef prepared a delicious three-course meal.",
    "The team won the championship last season.",
    "A rainbow appeared after the heavy rain.",
    "The firefighters bravely saved the family.",
    "She bought a bouquet of fresh flowers.",
    "The mechanic fixed the car engine quickly.",
    "We took a boat ride across the lake.",
    "The baby giggled when I tickled her feet.",
    "My father taught me how to ride a bicycle.",
    "The children built a sandcastle on the beach.",
    "I prefer tea over coffee in the morning.",
    "The dancer performed with grace and elegance.",
    "Our vacation to Italy was unforgettable.",
    "She writes poetry in her free time.",
    "We enjoyed watching the fireworks display.",
    "The mountain peak was covered in snow.",
    "The astronaut described life in space.",
    "They watched the sunrise from the hilltop.",
    "The athlete trained hard for the marathon.",
    "The detective solved the mystery case.",
    "She baked a chocolate cake for dessert.",
    "The garden was filled with colorful butterflies.",
    "They played chess by the fireplace.",
    "The ancient ruins were fascinating to explore.",
    "She wore a beautiful silver necklace.",
    "He composed a symphony for the orchestra.",
    "The scientist discovered a new planet.",
    "We went camping under the starry sky.",
    "The knight fought bravely in battle.",
    "She enjoys long walks in the countryside.",
    "The magician performed an amazing trick.",
    "The violinist played a mesmerizing melody.",
    "We adopted a kitten from the shelter.",
    "The festival was full of joy and laughter.",
    "I am going to the store later.",
    "This is a simple test sentence.",
    "She likes to read books.",
    "He is reading a book.",
    "I love eating apples.",
    "The cat is sitting on the mat.",
    "I will meet you at the park.",
    "They're going to the mall.",
    "I know the answer.",
    "The weather is nice today.",
    "I just want to say hello.",
    "This is a great example.",
    "He is a great teacher.",
    "She bought a new phone.",
    "I enjoy doing sport every day.",
    "She was dyeing her hair.",
    "He has a strong feeling about this.",
    "Their house is very big.",
    "They're going to the store.",
    "I am trying to study.",
    "The car doesn't work.",
    "He was very happy.",
    "She does not like coffee.",
    "This is an interesting book.",
    "Can you believe it?",
    "He tried to help.",
    "We finally arrived.",
    "This is a beautiful day.",
    "She really wants to go.",
    "Let's go to the restaurant.",
    "She gave me a gift.",
    "He drove too fast.",
    "I think we will win.",
    "The doctor prescribed medicine.",
    "I will definitely call you.",
    "She always smiles.",
    "He committed a mistake.",
    "She surprised everyone.",
    "You're welcome!",
    "Their car is faster than ours.",
    "I didn't see him.",
    "She signed the contract.",
    "The children are playing outside.",
    "Let's meet at 5 PM.",
    "He is responsible for this.",
    "She is a beautiful singer.",
    "This is a difficult question.",
    "We finally finished our work.",
    "I hope you're doing well.",
    "His advice was very helpful."
]

test_cases = [(f'{introduce_errors(sentence)}', f'{sentence}') for sentence in correct_sentences]


def save_test_cases(output_csv="testing_sentences.csv"):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["incorrect", "correct"])
        writer.writerows(test_cases)

    print(f"Generated {len(test_cases)} test cases and saved to {output_csv}")

save_test_cases()

Generated 130 test cases and saved to testing_sentences.csv


In [13]:
import csv


def test_correction(csv_filename):
    num_sentences = 0
    correct_sencences = 0
    with open(csv_filename, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)
        for incorrect, correct in reader:
            num_sentences +=1
            predicted = correct_sentence(incorrect, unigram_counts, bigram_counts, trigram_counts, fivegram_counts)
            # At this moment I don't know how to fix, that at the end of the sentence I got space before ".". So I check accuracy like this ;)
            predicted = predicted.replace(' .', '.')
            if predicted == correct:
                correct_sencences += 1
            print(f"Input: {incorrect}\nExpected: {correct}\nPredicted: {predicted}\n")
        print(f"Accuracy: {correct_sencences/num_sentences}")


test_correction("testing_sentences.csv")

Input: I am going too the store later.
Expected: I am going to the store later.
Predicted: I am going too the store later.

Input: is This a simple test sentence.
Expected: This is a simple test sentence.
Predicted: is This a simple test sentence.

Input: enjoys She playing the piano.
Expected: She enjoys playing the piano.
Predicted: enjoys She playing the piano.

Input: weather The is really nice today.
Expected: The weather is really nice today.
Predicted: weather The is really nice today.

Input: need We to finish our project soon.
Expected: We need to finish our project soon.
Predicted: need We to finish our project soon.

Input: He quickly ran across the street to catch the bus.
Expected: He quickly ran across the street to catch the bus.
Predicted: He quickly ran across the street to catch the bus.

Input: My favorite color is blue, but I also lik red.
Expected: My favorite color is blue, but I also like red.
Predicted: My favorite color is blue , but I also like red.

Input: do

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)